In [2]:
from unstructured.partition.auto import partition
from unstructured.chunking.basic import chunk_elements
from unstructured.chunking.title import chunk_by_title
import pandas as pd

In [3]:
text_pdf_filename = "samples/layout-parser-paper.pdf"
table_img_pdf_filename = "samples/embedded-images-tables.pdf"
img_pdf_filename = "samples/embedded-images.pdf"
img_filename = "samples/layout-parser-paper-fast.jpg"

prodesa_alameda_pdf_filename = "samples/prodesa/memoria-alameda-del-rio.pdf"

In [3]:
elements = partition(filename=text_pdf_filename, strategy='hi_res')
# chunks_el = chunk_elements(elements, max_characters=1024, overlap=204)

/home/parjua/.pyenv/versions/3.11.0/envs/unstructured-parser/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [4]:
print("Number of elements:", len(elements))
# print("Number of chunks:", len(chunks_el))

# max([len(chunk.text) for chunk in chunks_el])



Number of elements: 178


In [5]:
image_elememts = partition(filename=text_pdf_filename)

In [6]:
pdf_table_elements = partition(filename=table_img_pdf_filename)

In [42]:
img_pdf_elements = partition(filename=img_pdf_filename)

In [4]:
prodesa_alameda_elements = partition(filename=prodesa_alameda_pdf_filename, strategy='hi_res', pdf_infer_table_structure=True)

The pdf_infer_table_structure kwarg is deprecated. Please use skip_infer_table_types instead.
/home/parjua/.pyenv/versions/3.11.0/envs/unstructured-parser/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

In [24]:
prodesa_alameda_chunks = chunk_by_title(prodesa_alameda_elements, max_characters=15024, overlap=204)

In [12]:
print(prodesa_alameda_chunks[1].text)

% A VIVIENDA A TERRAZAS AREA TOTAL UNIDADES APTO TIPO A 75,35 4,95 80,30 180 APTO TIPO B 65,35 5,02 70,37 171 APTO TIPO C 56,57 5,02 61,59 9 APTO TIPO D 58,54 0,00 58,54 60 APTO TIPO D1 58,69 0,00 58,69 30 APTO TIPO D2 58,89 0,00 58,89 2 APTO TIPO D3 58,59 0,00 58,59 28 APTO TIPO D4 58,77 0,00 58,77 28 APTO TIPO E 49,51 0,00 49,51 2


In [25]:
len(prodesa_alameda_chunks)

7

## Parsing

In [174]:
import requests
import json

url = "http://localhost:8081"
headers = {
    "Content-Type": "application/json"
}

def get_schema(json_nodes, text, schema, outputs=["extraction", "res"], **kwargs):
    data = {
        "flow_id": "LLM_FLW_01HSEZDCTRGPSX11DDWH21K0XX",
        "outputs": outputs,
        "custom_values": {
            "language": "ES_CO",
            "identified_nodes": json_nodes,
            "optional_schema": schema,
            "contextual_text": text,
            **kwargs
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(data)).json()["result"]["user_defined_outputs"]
    print(response)

    parsed_value = response[1]["value"]
    # parsed_extraction = response[0]["value"]

    # print(parsed_extraction)
    nodes = parsed_value["entities"]
    relations = parsed_value["relations"]

    nodes_df = pd.DataFrame(nodes)
    relations_df = pd.DataFrame(relations)

    return nodes_df, relations_df

def merge_schemas(df1, df2, join_indices):
    df_merged = pd.merge(df1, df2, on=join_indices, how='outer', suffixes=('_left', '_right'))

    df_merged['props'] = df_merged.apply(
        lambda row: {**(row['props_left'] if isinstance(row['props_left'], dict) else {}),
                     **(row['props_right'] if isinstance(row['props_right'], dict) else {})},
        axis=1
    )

    df_merged['type'] = df_merged.apply(
        lambda row: row['type_left'] if not pd.isna(row['type_left']) else row['type_right'],
        axis=1
    )

    df_final = df_merged[join_indices + ['type', 'props']]

    return df_final

In [144]:
nodes_df, relations_df = None, None
count = 0

for chunk in prodesa_alameda_chunks:
    json_nodes = ""
    if nodes_df is not None:
        json_nodes = nodes_df.to_json(orient="records")

    _nodes_df, _relations_df = get_schema(json_nodes, chunk.text, "node=Ciudad;node=Macroproyecto;node=Proyecto;node=Tipo_Apto\nedge=HAS")

    if not _nodes_df.empty:
        _nodes_df['props'] = _nodes_df['props'].apply(lambda x: {v['name']: v['value'] for v in x})

    if not _relations_df.empty:
        _relations_df['props'] = _relations_df['props'].apply(lambda x: {v['name']: v['value'] for v in x})

    if nodes_df is None:
        nodes_df = _nodes_df

    if not _nodes_df.empty:
        nodes_df = merge_schemas(nodes_df, _nodes_df, ['entity_id'])

    if relations_df is None:
        relations_df = _relations_df

    if not _relations_df.empty:
        relations_df = merge_schemas(relations_df, _relations_df, ['from_entity_id', 'to_entity_id'])

[{'name': 'extraction', 'description': None, 'value': '# Extracted Entities and Relationships\n\n## Nodes\n\n1. **Ciudad**\n   - Nombre: Barranquilla\n   - Departamento: Atlántico\n\n2. **Macroproyecto**\n   - Nombre: Alameda del Rio\n   - Ubicación: Predios de "El Volador", Avenida Circunvalar, entre carreras 38 y 46\n   - Características:\n     - 92 hectáreas destinadas a espacio público\n     - 89 hectáreas de vivienda, comercio y equipamientos\n     - Capacidad para 22,000 familias\n     - 50 hectáreas de parques, alamedas, andenes y ciclorrutas\n\n3. **Proyecto**\n   - Nombre: Pardela\n   - Localización: M-R14 del Macroproyecto Alameda del Rio\n   - Estrato Socio-Económico: 3-4\n   - Área Construida Total: 61,358.83 m²\n   - Área de Vivienda: 35,817.56 m²\n   - Unidades de Vivienda: 510\n   - Torres: 4 (3 torres de 15 pisos con 8 apartamentos por piso, 1 torre de 15 pisos con 10 apartamentos por piso)\n   - Sistema Constructivo: Muros estructurales de concreto\n\n4. **Tipo_Apto**\

In [167]:
nodes_df

,entity_id,type,props
0,CIUDAD_BARRANQUILLA,CIUDAD,"{'nombre': 'Barranquilla', 'departamento': 'At..."
1,MACROPROYECTO_ALAMEDA_DEL_RIO,MACROPROYECTO,"{'nombre': 'Alameda del Rio', 'ubicacion': 'Pr..."
2,PROYECTO_PARDELA,PROYECTO,"{'nombre': 'Pardela', 'localizacion': 'M-R14 d..."
3,TIPO_APTO_TIPO_A,TIPO_APTO,"{'tipo': 'A', 'area': '80.0 m²', 'unidades': '..."
4,TIPO_APTO_TIPO_B,TIPO_APTO,"{'tipo': 'B', 'area': '70.0 m²', 'unidades': '..."
5,TIPO_APTO_TIPO_C,TIPO_APTO,"{'tipo': 'C', 'area_de_vivienda': '56.57 m²', ..."
6,TIPO_APTO_TIPO_D,TIPO_APTO,"{'tipo': 'D', 'area': '57.0 m²', 'unidades': '..."
7,TIPO_APTO_TIPO_D1,TIPO_APTO,"{'tipo': 'D1', 'area_de_vivienda': '58.69 m²',..."
8,TIPO_APTO_TIPO_D2,TIPO_APTO,"{'tipo': 'D2', 'area_de_vivienda': '58.89 m²',..."
9,TIPO_APTO_TIPO_D3,TIPO_APTO,"{'tipo': 'D3', 'area_de_vivienda': '58.59 m²',..."


In [153]:
relations_df



,from_entity_id,to_entity_id,type,props
0,CIUDAD_BARRANQUILLA,MACROPROYECTO_ALAMEDA_DEL_RIO,HAS,{'relacion': 'La ciudad de Barranquilla contie...
1,MACROPROYECTO_ALAMEDA_DEL_RIO,PROYECTO_PARDELA,HAS,{'relacion': 'El Macroproyecto Alameda del Rio...
2,PROYECTO_PARDELA,TIPO_APTO_TIPO_A,HAS,{'relacion': 'El Proyecto Pardela ofrece apart...
3,PROYECTO_PARDELA,TIPO_APTO_TIPO_B,HAS,{'relacion': 'El Proyecto Pardela ofrece apart...
4,PROYECTO_PARDELA,TIPO_APTO_TIPO_C,HAS,{'relacion': 'El Proyecto Pardela ofrece apart...
5,PROYECTO_PARDELA,TIPO_APTO_TIPO_D,HAS,{'relacion': 'El Proyecto Pardela ofrece apart...
6,PROYECTO_PARDELA,TIPO_APTO_TIPO_D1,HAS,{'relacion': 'El Proyecto Pardela ofrece apart...
7,PROYECTO_PARDELA,TIPO_APTO_TIPO_D2,HAS,{'relacion': 'El Proyecto Pardela ofrece apart...
8,PROYECTO_PARDELA,TIPO_APTO_TIPO_D3,HAS,{'relacion': 'El Proyecto Pardela ofrece apart...
9,PROYECTO_PARDELA,TIPO_APTO_TIPO_D4,HAS,{'relacion': 'El Proyecto Pardela ofrece apart...


# CSV Loading

In [169]:
def create_csv(nodes_df, relations_df, filename):
    nodes_df.to_csv(filename + "_nodes.csv", index=False)
    relations_df.to_csv(filename + "_relations.csv", index=False)

In [171]:
create_csv(nodes_df, relations_df, "outputs/prodesa_alameda")

# Load into Neo4J

In [172]:
import dotenv
import os
from neo4j import GraphDatabase

load_status = dotenv.load_dotenv("Neo4j-0c2c4bc6-Created-2024-09-16.txt")
if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [225]:
import ast

def create_nodes_merges(file_path):
    df = pd.read_csv(file_path)

    props_by_type = {}

    for _, row in df.iterrows():
        props = ast.literal_eval(row['props'])
        props_by_type[row['type']] = props_by_type.get(row["type"], set()).union(set(props.keys()))

    merge_by_type = {}

    for key, value in props_by_type.items():
        merge_by_type[key] = f"MERGE (n:{key} {{{', '.join([f'{k}: row.{k}' for k in value])}}})"

    return merge_by_type


In [226]:
merges = create_nodes_merges("outputs/prodesa_alameda_nodes.csv")

print(merges)

with driver.session() as session:
    for node_type, merge_str in merges.items():
        session.run(f"""
        LOAD CSV WITH HEADERS FROM 'outputs/prodesa_alameda_nodes.csv'
        AS row
        {merge_str}""")

{'CIUDAD': 'MERGE (n:CIUDAD {nombre: row.nombre, departamento: row.departamento})', 'MACROPROYECTO': 'MERGE (n:MACROPROYECTO {ubicacion: row.ubicacion, nombre: row.nombre, caracteristicas: row.caracteristicas})', 'PROYECTO': 'MERGE (n:PROYECTO {localizacion: row.localizacion, proporcion_de_parqueaderos: row.proporcion_de_parqueaderos, cupos_de_parqueo_motos: row.cupos_de_parqueo_motos, area_de_vivienda: row.area_de_vivienda, edificio_de_parqueos: row.edificio_de_parqueos, cupos_de_parqueo_bicicletas: row.cupos_de_parqueo_bicicletas, edificio_de_porteria_y_servicios_comunales: row.edificio_de_porteria_y_servicios_comunales, cupos_de_parqueo_visitantes: row.cupos_de_parqueo_visitantes, torres: row.torres, altura_libre_torres_a_b_y_c: row.altura_libre_torres_a_b_y_c, altura_libre_torre_d: row.altura_libre_torre_d, nombre: row.nombre, area_construida_total: row.area_construida_total, unidades_de_vivienda: row.unidades_de_vivienda, sistema_constructivo: row.sistema_constructivo, edificio_co

/tmp/ipykernel_15573/254020572.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


ClientError: {code: Neo.ClientError.Statement.ExternalResourceFailed} {message: Invalid URL 'outputs/prodesa_alameda_nodes.csv': no protocol: outputs/prodesa_alameda_nodes.csv ()}